# Install dependencies


In [1]:
!pip install -qU torch==2.9.0 torchvision==0.24.0 "ray[client,train]"==2.51.1

# Import dependencies


In [2]:
import os
from typing import Dict

import torch
from filelock import FileLock
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Normalize, ToTensor

import ray
import ray.train
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer

# Define functions


In [3]:
def get_dataloaders(batch_size):
    # Transform to normalize the input images
    transform = transforms.Compose([ToTensor(), Normalize((0.5,), (0.5,))])

    with FileLock(os.path.expanduser("~/data.lock")):
        # Download training data from open datasets
        training_data = datasets.FashionMNIST(
            root="~/data",
            train=True,
            download=True,
            transform=transform,
        )

        # Download test data from open datasets
        test_data = datasets.FashionMNIST(
            root="~/data",
            train=False,
            download=True,
            transform=transform,
        )

    # Create data loaders
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader


# Model Definition
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_func_per_worker(config: Dict):
    lr = config["lr"]
    epochs = config["epochs"]
    batch_size = config["batch_size_per_worker"]

    # Get dataloaders inside the worker training function
    train_dataloader, test_dataloader = get_dataloaders(batch_size=batch_size)

    # [1] Prepare Dataloader for distributed training
    # Shard the datasets among workers and move batches to the correct device
    # =======================================================================
    train_dataloader = ray.train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = ray.train.torch.prepare_data_loader(test_dataloader)

    model = NeuralNetwork()

    # [2] Prepare and wrap your model with DistributedDataParallel
    # Move the model to the correct GPU/CPU device
    # ============================================================
    model = ray.train.torch.prepare_model(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Model training loop
    for epoch in range(epochs):
        if ray.train.get_context().get_world_size() > 1:
            # Required for the distributed sampler to shuffle properly across epochs.
            train_dataloader.sampler.set_epoch(epoch)

        model.train()
        print(f"Train Epoch {epoch}")
        for X, y in train_dataloader:
            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        test_loss, num_correct, num_total = 0, 0, 0
        with torch.no_grad():
            print(f"Test Epoch {epoch}")
            for X, y in test_dataloader:
                pred = model(X)
                loss = loss_fn(pred, y)

                test_loss += loss.item()
                num_total += y.shape[0]
                num_correct += (pred.argmax(1) == y).sum().item()

        test_loss /= len(test_dataloader)
        accuracy = num_correct / num_total

        # [3] Report metrics to Ray Train
        # ===============================
        ray.train.report(metrics={"loss": test_loss, "accuracy": accuracy})


@ray.remote
def train_fashion_mnist(num_workers=5, use_gpu=False):
    global_batch_size = 32

    train_config = {
        "lr": 1e-3,
        "epochs": 10,
        "batch_size_per_worker": global_batch_size // num_workers,
    }

    # Configure computation resources
    scaling_config = ScalingConfig(num_workers=num_workers, use_gpu=use_gpu)

    # Initialize a Ray TorchTrainer
    trainer = TorchTrainer(
        train_loop_per_worker=train_func_per_worker,
        train_loop_config=train_config,
        scaling_config=scaling_config,
    )

    # [4] Start distributed training
    # Run `train_func_per_worker` on all workers
    # =============================================
    result = trainer.fit()
    print(f"Training result: {result}")

# Connect to Ray Cluster


In [4]:
ray.init(
    address="ray://raycluster-gvt-head-svc.default.svc.cluster.local:10001",
    runtime_env={"pip": ["torch==2.9.0", "torchvision==0.24.0"]},
)

2025-11-11 07:36:36,748	INFO client_builder.py:241 -- Passing the following kwargs to ray.init() on the server: log_to_driver
SIGTERM handler is not set because current thread is not the main thread.
2025-11-11 07:37:07,143	WARNING utils.py:1318 -- Python patch version mismatch: The cluster was started with:
    Ray: 2.51.1
    Python: 3.11.11
This process on Ray Client was started with:
    Ray: 2.51.1
    Python: 3.11.6



Python version:,3.11.11
Ray version:,2.51.1
Dashboard:,http://10.0.3.101:8265


# Train model remotely


In [5]:
ray.get(train_fashion_mnist.remote())

(autoscaler +8s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +8s) Adding 1 node(s) of type gvt-group.
(autoscaler +8s) Resized to 5 CPUs.


(TrainController pid=412, ip=10.0.2.213) Attempting to start training worker group of size 5 with the following resources: [{'CPU': 1}] * 5


(RayTrainWorker pid=546, ip=10.0.2.213) [Gloo] Rank 0 is connected to 4 peer ranks. Expected number of connected peer ranks is : 4
(RayTrainWorker pid=549, ip=10.0.2.213) [Gloo] Rank 1 is connected to 4 peer ranks. Expected number of connected peer ranks is : 4
(RayTrainWorker pid=547, ip=10.0.2.213) [Gloo] Rank 2 is connected to 4 peer ranks. Expected number of connected peer ranks is : 4
(RayTrainWorker pid=548, ip=10.0.2.213) [Gloo] Rank 3 is connected to 4 peer ranks. Expected number of connected peer ranks is : 4
(RayTrainWorker pid=641, ip=10.0.2.213) [Gloo] Rank 4 is connected to 4 peer ranks. Expected number of connected peer ranks is : 4


(RayTrainWorker pid=546, ip=10.0.2.213) Setting up process group for: env:// [rank=0, world_size=5]
(TrainController pid=412, ip=10.0.2.213) Started training worker group of size 5: 
(TrainController pid=412, ip=10.0.2.213) - (ip=10.0.2.213, pid=546) world_rank=0, local_rank=0, node_rank=0
(TrainController pid=412, ip=10.0.2.213) - (ip=10.0.2.213, pid=549) world_rank=1, local_rank=1, node_rank=0
(TrainController pid=412, ip=10.0.2.213) - (ip=10.0.2.213, pid=547) world_rank=2, local_rank=2, node_rank=0
(TrainController pid=412, ip=10.0.2.213) - (ip=10.0.2.213, pid=548) world_rank=3, local_rank=3, node_rank=0
(TrainController pid=412, ip=10.0.2.213) - (ip=10.0.2.213, pid=641) world_rank=4, local_rank=4, node_rank=0
  0%|          | 0.00/26.4M [00:00<?, ?B/s]) 
  0%|          | 32.8k/26.4M [00:00<02:19, 189kB/s]
  0%|          | 65.5k/26.4M [00:00<02:20, 188kB/s]
  0%|          | 131k/26.4M [00:00<01:36, 273kB/s] 
  1%|          | 197k/26.4M [00:00<01:23, 313kB/s]
  1%|▏         | 360k/26

# Shut down Ray workers


In [6]:
ray.shutdown()

(train_fashion_mnist pid=287, ip=10.0.2.213) Training result: Result(metrics=None, checkpoint=None, error=None, path='/home/ray/ray_results/ray_train_run-2025-11-10_23-39-29', metrics_dataframe=None, best_checkpoints=[], _storage_filesystem=<pyarrow._fs.LocalFileSystem object at 0xfffd46bdd7f0>)
